<a href="https://colab.research.google.com/github/rahulthaker/Sevpuripow-AI/blob/main/flux_execution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#@title Huggingface login & Gemini API
from huggingface_hub import login
login()

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
#@title Create Project Folder
import os
new_folder_path = "/content/drive/My Drive/FLUX1"

# Create the folder if it doesn't exist
if not os.path.exists(new_folder_path):
    os.makedirs(new_folder_path)

In [5]:
#@title Install Required Packages, Import Libraries, Setup Pipeline, and Load LoRA

!pip install git+https://github.com/huggingface/diffusers.git --upgrade
!pip install transformers accelerate --quiet
!pip install git+https://github.com/huggingface/peft.git
!pip install bitsandbytes safetensors
#!pip install google-generativeai

import os
import re
import random
import datetime
import torch
import gc
import itertools
from tqdm import tqdm
from IPython.display import display
from diffusers import FluxPipeline
from PIL import Image


# Set up the API key

# Setup device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load the FLUX pipeline
pipe = FluxPipeline.from_pretrained(
    "black-forest-labs/FLUX.1-dev",
    torch_dtype=torch.float16,
    use_safetensors=True,
)

# Move the pipeline to GPU and enable optimizations
pipe = pipe.to(device)
pipe.enable_attention_slicing()
pipe.vae.enable_slicing()
pipe.vae.enable_tiling()

# Optimize tokenizer
pipe.tokenizer = pipe.tokenizer.from_pretrained(pipe.tokenizer.name_or_path, use_fast=True)

print("Pipeline loaded and moved to device.")
print("Memory optimization techniques enabled.")
print("Tokenizer optimized.")


  Cloning https://github.com/huggingface/diffusers.git to /tmp/pip-req-build-3olg5jn0
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/diffusers.git /tmp/pip-req-build-3olg5jn0
  Resolved https://github.com/huggingface/diffusers.git to commit aa73072f1f7014635e3de916cbcf47858f4c37a0
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/huggingface/peft.git to /tmp/pip-req-build-zppfp4_p
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft.git /tmp/pip-req-build-zppfp4_p
  Resolved https://github.com/huggingface/peft.git to commit 5d944589d24a85ea21c8ff71e06c255c6b01378b
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Using device: cuda


model_index.json:   0%|          | 0.00/536 [00:00<?, ?B/s]

Fetching 23 files:   0%|          | 0/23 [00:00<?, ?it/s]

scheduler/scheduler_config.json:   0%|          | 0.00/273 [00:00<?, ?B/s]

tokenizer/merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

...

diffusion_pytorch_model.safetensors:   0%|          | 0.00/168M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers


Pipeline loaded and moved to device.
Memory optimization techniques enabled.
Tokenizer optimized.


In [6]:
def show_images(images):
    for image in images:
        display(image)

def save_images(images, folder_path, caption, seed, start_index=0, method="", enhanced=False, format_suffix=""):
    truncated_caption = re.sub(r'[\/*?:"<>|]', "", caption)[:200]
    for idx, image in enumerate(images):
        enh_suffix = "_ENH" if enhanced else ""
        filename = f"{truncated_caption}{seed}{method}_img{start_index + idx + 1}{format_suffix}{enh_suffix}.png"
        image.save(os.path.join(folder_path, filename))

def save_settings(filename, dynamic_caption, processed_caption, enhanced_caption, guidance_scale, num_inference_steps, seed, height1, width1, height2, width2, lora_info=None):
    settings = f"""
Dynamic Prompt: {dynamic_caption}

Processed Prompt: {processed_caption}

Enhanced Prompt: {enhanced_caption}

Guidance Scale: {guidance_scale}

Num Inference Steps: {num_inference_steps}

Seed: {seed}

Dimensions Image 1: {width1}x{height1}
Dimensions Image 2: {width2}x{height2}

LoRA Information:
"""
    if lora_info:
        for lora_name, strength in lora_info.items():
            settings += f"{lora_name}: Strength {strength}\n"
    else:
        settings += "No LoRAs applied\n"

    with open(filename, 'w') as f:
        f.write(settings.strip())

def count_combinations(prompt, enhance_with_gemini=False):
    sequence_pattern = re.compile(r'\[(.*?)\]')
    sequences = sequence_pattern.findall(prompt)
    if not sequences:
        return 2 if enhance_with_gemini else 1

    options_counts = [len(seq.split('|')) for seq in sequences]
    total_combinations = 1
    for count in options_counts:
        total_combinations *= count

    return total_combinations * 2 if enhance_with_gemini else total_combinations

def process_sequential_prompt(prompt):
    sequence_pattern = re.compile(r'\[(.*?)\]')
    sequences = sequence_pattern.findall(prompt)
    if not sequences:
        return [prompt]
    sequence_options = [seq.split('|') for seq in sequences]
    combinations = list(itertools.product(*sequence_options))
    result_prompts = []
    for combo in combinations:
        new_prompt = prompt
        for i, option in enumerate(combo):
            new_prompt = new_prompt.replace(f'[{sequences[i]}]', option.strip(), 1)
        result_prompts.append(new_prompt)
    return result_prompts

def process_dynamic_prompt(prompt):
    def replace_placeholder(match):
        choices = match.group(1).split("|")
        return random.choice(choices).strip()
    dynamic_prompt_pattern = re.compile(r"\{([^{}]*)\}")
    while dynamic_prompt_pattern.search(prompt):
        prompt = dynamic_prompt_pattern.sub(replace_placeholder, prompt)
    return prompt

def enhance_prompt_with_gemini(prompt):
    model = genai.GenerativeModel('gemini-pro')
    response = model.generate_content(gemini_prompt + "\n\nOriginal prompt: " + prompt)
    return response.text.strip()

In [7]:
#@title Define Main Function
def generate_save_display_images(pipe, num_images_to_generate, width1, height1, width2, height2, num_inference_steps, start_index=0, caption="", seed=None, variations=0, lora_config=None, enhance_with_gemini=False):
    if lora_config:
        try:
            pipe.unload_lora_weights()  # Unload any previously loaded LoRA weights
            for lora in lora_config:
                pipe.load_lora_weights(lora['path'], adapter_name=lora['name'])

            lora_names = [lora['name'] for lora in lora_config]
            lora_scales = [lora['alpha'] for lora in lora_config]

            pipe.set_adapters(lora_names, adapter_weights=lora_scales)
            print(f"LoRAs applied: {lora_names} with weights {lora_scales}")
        except Exception as e:
            print(f"Error applying LoRAs: {str(e)}")
            print("Continuing without LoRAs...")
    else:
        print("No LoRAs applied")

    if '[' in caption:
        num_combinations = count_combinations(caption, enhance_with_gemini)
        prompts = process_sequential_prompt(caption)
        print(f"Number of combinations: {num_combinations}")
        total_image_sets = num_combinations
    else:
        prompts = [caption] * num_images_to_generate
        total_image_sets = num_images_to_generate

    print(f"Total image sets to be generated: {total_image_sets}")
    print(f"Variations per set: {variations + 1}")
    print(f"Total images to be generated: {total_image_sets * (variations + 1) * 2 * (2 if enhance_with_gemini else 1)}")

    for idx in tqdm(range(len(prompts)), desc="Generating image sets"):
        base_prompt = prompts[idx]
        processed_caption = process_dynamic_prompt(base_prompt)

        print(f"\nGenerating image set {idx+1}/{total_image_sets} with prompt: {processed_caption}")

        # Generate images with original prompt
        generate_images(pipe, processed_caption, variations, width1, height1, width2, height2, num_inference_steps, seed, start_index, idx, save_folder_path, base_prompt, enhanced=False, enhanced_caption=None, lora_config=lora_config)

        # If Gemini enhancement is enabled, generate enhanced images
        if enhance_with_gemini:
            enhanced_caption = enhance_prompt_with_gemini(processed_caption)
            print(f"\nGenerating enhanced image set {idx+1}/{total_image_sets} with prompt: {enhanced_caption}")
            generate_images(pipe, enhanced_caption, variations, width1, height1, width2, height2, num_inference_steps, seed, start_index, idx, save_folder_path, base_prompt, enhanced=True, enhanced_caption=enhanced_caption, lora_config=lora_config)

        if (idx + 1) % 5 == 0:
            torch.cuda.empty_cache()
            gc.collect()

    print(f"\nGenerated and saved {total_image_sets * (variations + 1) * 2 * (2 if enhance_with_gemini else 1)} images.")

    # Unload LoRA weights after generation
    pipe.unload_lora_weights()
    print("LoRA weights unloaded")

    return pipe

def generate_images(pipe, processed_caption, variations, width1, height1, width2, height2, num_inference_steps, seed, start_index, idx, save_folder_path, original_caption, enhanced=False, enhanced_caption=None, lora_config=None):
    for v in range(variations + 1):
        local_seed = seed if seed is not None and seed != 0 else random.randint(0, 2**32 - 1)
        generator = torch.Generator("cuda").manual_seed(local_seed)

        print(f"Generating variation {v+1}/{variations+1}")

        for i in range(2):  # Two dimensions
            try:
                image = pipe(
                    prompt=processed_caption,
                    num_inference_steps=num_inference_steps,
                    guidance_scale=guidance_scale,
                    width=width1 if i == 0 else width2,
                    height=height1 if i == 0 else height2,
                    generator=generator,
                    max_sequence_length=512
                ).images[0]

                cleaned_caption = processed_caption.replace("Dynamic Prompt: ", "", 1)
                cleaned_caption = re.sub(r'[\/*?:"<>|]', '', cleaned_caption)[:200]
                save_images([image], save_folder_path, cleaned_caption, local_seed, start_index + idx + 1, enhanced=enhanced, format_suffix=f"_F{i+1}")
                print(f"Saved image: {cleaned_caption}_{local_seed}_var{v+1}_of{variations+1}_F{i+1}{'_ENH' if enhanced else ''}.png")

            except Exception as e:
                print(f"An error occurred during image generation: {str(e)}")
                print("Skipping this image and continuing...")
                continue

        timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        settings_filename = os.path.join(save_folder_path, f"settings_{timestamp}_img{start_index + idx + 1}_var{v+1}_of{variations+1}{'_ENH' if enhanced else ''}.txt")
        save_settings(settings_filename, original_caption, processed_caption, enhanced_caption, guidance_scale=guidance_scale,
                      num_inference_steps=num_inference_steps, seed=local_seed,
                      height1=height1, width1=width1, height2=height2, width2=width2,
                      lora_info={lora['name']: lora['alpha'] for lora in lora_config} if lora_config else None)

In [11]:
#@title LORA loading and configuration
use_lora1 = False #@param {type:"boolean"}
lora_path1 = "/content/drive/MyDrive/sevpuripow_Results/sevpuripow_lora_v1/sevpuripow_lora_v1.safetensors" #@param {type:"string"}
lora_strength1 = 1 #@param {type:"slider", min:0.0, max:1.0, step:0.1}


lora_config = []

def add_lora(lora_path, strength):
    if os.path.exists(lora_path):
        lora_name = os.path.splitext(os.path.basename(lora_path))[0]
        lora_config.append({"name": lora_name, "path": lora_path, "alpha": strength})
        print(f"LoRA added to configuration: {lora_name} (Strength: {strength})")
    else:
        print(f"LoRA file not found: {lora_path}")

# Add LoRAs to configuration
add_lora(lora_path1, lora_strength1)


if not lora_config:
    print("No LoRAs added to configuration.")
else:
    print(f"Added {len(lora_config)} LoRAs to configuration.")

print("\nLoRA Configuration:")
for lora in lora_config:
    print(f"  - {lora['name']}: Strength {lora['alpha']}")

# Optional: Empty cache to free up memory
torch.cuda.empty_cache()
gc.collect()

LoRA added to configuration: sevpuripow_lora_v1 (Strength: 1)
Added 1 LoRAs to configuration.

LoRA Configuration:
  - sevpuripow_lora_v1: Strength 1


202

In [18]:
#@title Settings

#@markdown Save Location Settings
custom_save_path = "/content/drive/MyDrive/FLUX1" #@param {type:"string"}

#@markdown General Settings (number of images & prompt)
num_images_to_generate = 5 #@param {type:"number", min:1, max:10000, step:1}
variations = 0 #@param {type:"slider", min:0, max:10, step:1}


#@markdown The prompt will concatenate the 4 captions:
caption1 = "sevpuripow man as superman" #@param {type:"string"}
caption2 = "" #@param {type:"string"}
caption3 = "" #@param {type:"string"}
caption4 = "" #@param {type:"string"}
caption = " ".join([c for c in [caption1, caption2, caption3, caption4] if c])
reservoir = "not used" #@param {type:"string"}


#@markdown Seed for Reproducibility (0 = random)
seed = 0 #@param {type:"number"}

#@markdown Guidance Scale (default: 3.5 - but 2 is way more realistic for photo)
guidance_scale = 2.5 #@param {type:"slider", min:0.0, max:20.0, step:0.1}

#@markdown Number of Inference Steps (default: 50)
num_inference_steps = 35 #@param {type:"slider", min:1, max:100, step:1}

#@markdown Image Dimensions for First Image
width1 = 1024 #@param {type:"slider", min:512, max:3072, step:64}
height1 = 1280 #@param {type:"slider", min:512, max:3072, step:64}

#@markdown Image Dimensions for Second Image
width2 = 1536 #@param {type:"slider", min:512, max:3072, step:64}
height2 = 1024 #@param {type:"slider", min:512, max:3072, step:64}

#@markdown Enhance prompts with Gemini for additional images
enhance_with_gemini = False #@param {type:"boolean"}

# Set up save folder
if custom_save_path:
    save_folder_path = os.path.join("/content/drive/MyDrive", custom_save_path)
else:
    save_folder_path = "/content/drive/MyDrive/FLUX1"

os.makedirs(save_folder_path, exist_ok=True)

In [17]:
#@title Generate Images
print("Generating image pairs using FLUX.1-dev model with LoRA...")
pipe = generate_save_display_images(
    pipe,
    num_images_to_generate,
    width1,
    height1,
    width2,
    height2,
    num_inference_steps,
    start_index=0,
    caption=caption,
    seed=seed,
    variations=variations,
    lora_config=lora_config,
    enhance_with_gemini=enhance_with_gemini
)

print("Image generation complete.")

# Optional: Display a sample image
if 'image' in locals():
    display(image)
else:
    print("No sample image available to display.")

# Print total number of images generated
total_images = num_images_to_generate * (variations + 1) * 2
if enhance_with_gemini:
    total_images *= 2
print(f"Total images generated: {total_images}")

# Print save location
print(f"Images saved in: {save_folder_path}")

# Optional: Print LoRA information
if lora_config:
    print("LoRA configurations used:")
    for lora in lora_config:
        print(f"  - {lora['name']}: Strength {lora['alpha']}")
else:
    print("No LoRAs were applied.")

# Optional: Clean up
torch.cuda.empty_cache()
gc.collect()
print("Memory cleared.")

Generating image pairs using FLUX.1-dev model with LoRA...


Loading adapter weights from state_dict led to unexpected keys not found in the model:  ['transformer_blocks.0.attn.add_k_proj.lora_A.sevpuripow_lora_v1.weight', 'transformer_blocks.0.attn.add_k_proj.lora_B.sevpuripow_lora_v1.weight', 'transformer_blocks.0.attn.add_v_proj.lora_A.sevpuripow_lora_v1.weight', 'transformer_blocks.0.attn.add_v_proj.lora_B.sevpuripow_lora_v1.weight', 'transformer_blocks.0.attn.add_q_proj.lora_A.sevpuripow_lora_v1.weight', 'transformer_blocks.0.attn.add_q_proj.lora_B.sevpuripow_lora_v1.weight', 'transformer_blocks.1.attn.add_k_proj.lora_A.sevpuripow_lora_v1.weight', 'transformer_blocks.1.attn.add_k_proj.lora_B.sevpuripow_lora_v1.weight', 'transformer_blocks.1.attn.add_v_proj.lora_A.sevpuripow_lora_v1.weight', 'transformer_blocks.1.attn.add_v_proj.lora_B.sevpuripow_lora_v1.weight', 'transformer_blocks.1.attn.add_q_proj.lora_A.sevpuripow_lora_v1.weight', 'transformer_blocks.1.attn.add_q_proj.lora_B.sevpuripow_lora_v1.weight', 'transformer_blocks.2.attn.add_k_pr

LoRAs applied: ['sevpuripow_lora_v1'] with weights [1]
Total image sets to be generated: 10
Variations per set: 1
Total images to be generated: 20


Generating image sets:   0%|          | 0/10 [00:00<?, ?it/s]


Generating image set 1/10 with prompt: sevpuripow man as superman
Generating variation 1/1


...

  0%|          | 0/35 [00:00<?, ?it/s]

Saved image: sevpuripow man as superman_151135441_var1_of1_F1.png


  0%|          | 0/35 [00:00<?, ?it/s]

Generating image sets:  60%|██████    | 6/10 [06:45<04:30, 67.54s/it]


KeyboardInterrupt: 